In [1]:
import pandas as pd
import os
from data_cleaning import normalize_tracking, feature_engineering,get_position_count,label_run_or_pass
from data_loading import load_tracking_data, pivot_data,load_previous_year_data, load_stadium_data, load_weather_data
root_dir = os.getcwd()

games_fname= os.path.join(root_dir, "data/games.csv")
plays_fname = os.path.join(root_dir,  "data/plays.csv")
players_fname = os.path.join(root_dir,  "data/players.csv")
player_plays_fname = os.path.join(root_dir,  "data/player_play.csv")
tracking_fname_list_train = [os.path.join(root_dir,  f"data/tracking_week_{i}.csv") for i in range(1,9)]

In [2]:
df_games=pd.read_csv(games_fname)
df_games=pd.merge(df_games, load_stadium_data(),left_on='gameId', right_on='old_game_id', how='left')
df_games=pd.merge(df_games, load_weather_data(),on='gameId', how='left')

df_plays = feature_engineering(pd.read_csv(plays_fname))
df_tracking = pd.concat(
        [load_tracking_data(tracking_fname) for tracking_fname in tracking_fname_list_train]
    )
df_tracking=df_tracking[['gameId', 'playId', 'nflId','club' ,'o_standard', 'dir_standard', 'x_standard', 'y_standard', 's', 'a', 'dis']].groupby(['gameId', 'playId', 'nflId','club']).agg({'s':[ 'max',],'a':[ 'max'], 
    'o_standard':['mean', 'std'],'dis':['sum'],'dir_standard':['mean', 'std'], 'x_standard':['mean', 'std'], 'y_standard':['mean', 'std']}).reset_index()
df_tracking.columns=df_tracking.columns.map('|'.join).str.strip('|')
df_players = pd.read_csv(players_fname)
df_player_plays=pd.read_csv(player_plays_fname)
df_player_plays=pd.merge(df_player_plays, load_previous_year_data(2021), left_on='teamAbbr', right_on='team_abbr', how='outer')
    # aggregate plays, tracking, players tables
df_agg1=pd.merge(pd.merge(df_tracking, df_plays, left_on=[ 'gameId', 'playId','club'], right_on=[ 'gameId', 'playId', 'possessionTeam'], how='inner'), df_player_plays, left_on=[ 'gameId', 'playId', 'nflId'], right_on=[ 'gameId', 'playId', 'nflId'], how='left' )
df_agg2=pd.merge(df_agg1, df_games, on='gameId', how='inner')
df_final=pivot_data(label_run_or_pass(get_position_count(pd.merge(df_agg2, df_players, on='nflId', how='inner'))))
merged_id_df = df_final[['gameId','playId']].drop_duplicates()

2022 done.
Downcasting floats.


/Users/ccollins/Projects/NFL_BDB2025/.venv/lib/python3.9/site-packages/pandas/core/reshape/merge.py:1214: RuntimeWarning: invalid value encountered in cast
  if not (rk == rk.astype(lk.dtype))[~np.isnan(rk)].all():


INFO: Labeling plays as runs or passes


In [4]:
merged_id_df.to_csv('data/merged_ids.csv',index=False)